In [26]:
import os
import json
import random
random.seed(1)

In [27]:
with open("/data/nzxyin/GLOBE/preprocess_default/accent_clean_10.json", 'r') as f:
    filename_to_accent = json.load(f)

In [28]:
text_ids_dir = set(filename.split('.')[0] for filename in os.listdir("/data/nzxyin/GLOBE/preprocess_unilex/text_ids/"))
sparc_dir = set(filename.split('.')[0] for filename in os.listdir("/data/nzxyin/GLOBE/preprocess_unilex/sparc/"))

In [29]:
american_english = [k for k, v in filename_to_accent.items() if v == 'American English' and k.split('.')[0] in text_ids_dir and k.split('.')[0] in sparc_dir]
british_english = [k for k, v in filename_to_accent.items() if v == 'British English' and k.split('.')[0] in text_ids_dir and k.split('.')[0] in sparc_dir]

In [30]:
len(american_english), len(british_english)

(370139, 94277)

In [31]:
random.shuffle(american_english)
random.shuffle(british_english)

In [32]:
american_english=american_english[:len(british_english)]

In [33]:
american_train, american_dev, american_test = american_english[:-3500], american_english[-3500:-2500], american_english[-2500:]

In [34]:
british_train, british_dev, british_test = british_english[:-3500], british_english[-3500:-2500], british_english[-2500:]

In [35]:
train = {name: 0 for name in american_train} | {name: 1 for name in british_train}
dev = {name: 0 for name in american_dev} | {name: 1 for name in british_dev}
test = {name: 0 for name in american_test} | {name: 1 for name in british_test}

In [36]:
os.makedirs('/data/nzxyin/GLOBE/preprocess_unilex/splits/2_accent_splits', exist_ok=True)
with open('/data/nzxyin/GLOBE/preprocess_unilex/splits/2_accent_splits/train.json', 'w') as f:
    json.dump(train, f)
with open('/data/nzxyin/GLOBE/preprocess_unilex/splits/2_accent_splits/val.json', 'w') as f:
    json.dump(dev, f)
with open('/data/nzxyin/GLOBE/preprocess_unilex/splits/2_accent_splits/test.json', 'w') as f:
    json.dump(test, f)

In [29]:
import numpy as np

In [20]:
dur = np.load("/data/nzxyin/GLOBE/preprocess_default/durations/S_017790_103552.npy")
art = np.load("/data/nzxyin/GLOBE/preprocess_default/sparc/S_017790_103552.npy")

FileNotFoundError: [Errno 2] No such file or directory: '/data/nzxyin/GLOBE/preprocess_default/durations/S_017790_103552.npy'

In [57]:
dur

array([ 2,  9,  4,  9, 10,  1,  5,  2,  2,  5,  3,  2,  4,  3,  2,  3,  5,
        9,  4,  2,  6,  4,  5,  6,  5,  4,  6,  7,  3,  2,  3,  3,  4,  3,
        2,  2,  3,  2,  3,  5,  3,  6,  9,  2])

In [58]:
art.shape

(185, 15)

In [59]:
sum(dur), len(art)

(np.int64(184), 185)

In [43]:
620/155

4.0

In [ ]:
durations_dir = os.listdir("/data/nzxyin/GLOBE/preprocess_default/durations/")
text_ids_dir = os.listdir("/data/nzxyin/GLOBE/preprocess_default/text_ids/")
for filename in durations_dir:
    # assert filename in sparc_dir, print(filename)
    assert filename in text_ids_dir, print(filename)

Durations

In [19]:
from dataset import BetaBinomialInterpolator
import os
binomial_interpolator = BetaBinomialInterpolator()
text_ids_dir = "/data/nzxyin/GLOBE/preprocess_unilex/text_ids/" 
sparc_dir = "/data/nzxyin/GLOBE/preprocess_default/sparc/"
new_sparc_dir = "/data/nzxyin/GLOBE/preprocess_unilex/sparc/"
durations_dir = "/data/nzxyin/GLOBE/preprocess_unilex/durations/"
for filename in os.listdir(text_ids_dir):
    try:
        text, sparc = np.load(os.path.join(text_ids_dir, filename)), np.load(os.path.join(sparc_dir, filename))
        durations = binomial_interpolator(len(text), len(sparc))
        np.save(os.path.join(durations_dir, filename), sparc)
        np.save(os.path.join(new_sparc_dir, filename), durations)
    except FileNotFoundError as e:
        print("File", filename, 'does not exist')
    except Exception as e:
        print('File', filename, e)

File S_006004_007755.npy does not exist
File S_001429_048302.npy does not exist
File S_001940_282363.npy does not exist
File S_021784_401813.npy No data left in file
File S_011332_085549.npy does not exist


In [25]:
text = np.load(os.path.join(text_ids_dir, 'S_021784_401813.npy')) 
sparc = np.load(os.path.join(sparc_dir, 'S_021784_401813.npy'))
durations = binomial_interpolator(len(text), len(sparc))
np.save(os.path.join(durations_dir, filename), sparc)
np.save(os.path.join(new_sparc_dir, filename), durations)

Unilex

In [7]:
import os 
from symbols import get_id
import numpy as np
import json

In [8]:
text_dir = "/data/nzxyin/GLOBE/preprocess_unilex/text/"
text_ids_dir = "/data/nzxyin/GLOBE/preprocess_unilex/text_ids/"

In [23]:
reprocess_needed = []
# with open('reprocess_names.json') as f:
#     files = json.load(f)
files = ['S_021784_401813.txt']
for filename in files:
    with open(os.path.join(text_dir, filename)) as f:
        syms = f.read().strip().split(" ")
    # print(filename)
    try:
        ids = list(map(lambda x: get_id(x), syms))
        np.save(os.path.join(text_ids_dir, filename[:-3] + "npy"), np.array(ids))
    except KeyError as e:
        reprocess_needed.append(filename)

    

In [13]:
reprocess_needed

['S_007542_445396.txt',
 'S_009674_072422.txt',
 'S_023216_171072.txt',
 'S_014518_226656.txt',
 'S_018488_371696.txt',
 'S_006752_469961.txt',
 'S_007406_313856.txt',
 'S_009054_542656.txt',
 'S_010612_324147.txt',
 'S_015873_230279.txt',
 'S_013257_119526.txt',
 'S_008323_509609.txt',
 'S_000955_357688.txt',
 'S_009626_315404.txt',
 'S_016984_210858.txt',
 'S_018007_355876.txt',
 'S_014796_044016.txt',
 'S_011729_516480.txt',
 'S_008903_522621.txt',
 'S_006714_379829.txt',
 'S_012814_187626.txt',
 'S_015540_081512.txt',
 'S_008272_062339.txt',
 'S_002033_529623.txt',
 'S_018944_414563.txt',
 'S_004757_047081.txt',
 'S_019131_058965.txt',
 'S_016243_184539.txt',
 'S_015490_125923.txt',
 'S_014548_394497.txt',
 'S_014876_153160.txt',
 'S_007974_309566.txt',
 'S_011423_497161.txt',
 'S_005760_482797.txt',
 'S_012089_505094.txt',
 'S_007751_444143.txt',
 'S_022025_459374.txt',
 'S_020985_071684.txt',
 'S_014183_351849.txt',
 'S_012015_197230.txt',
 'S_017260_496781.txt',
 'S_020969_42748

In [16]:
import json
with open('reprocess_names.json', 'w') as f:
    json.dump(reprocess_needed, f)

In [14]:
len(reprocess_needed)

3014

In [5]:
import os
print(len(os.listdir('/data/nzxyin/GLOBE/preprocess_unilex/text_ids')))

579082
